In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset
from tqdm import tqdm

# 数据路径
train_dir = '/root/datasets/BTXRD3/train'
val_dir = '/root/datasets/BTXRD3/val'
output_dir = '/root/datasets/ResNet直接骨3分类/'

# 创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 数据预处理和加载
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)
class_names = full_dataset.classes

# 构建ResNet50模型
def build_model():
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
    model.fc = nn.Linear(model.fc.in_features, 3)
    return model

# 交叉验证和模型训练
kf = KFold(n_splits=5, shuffle=True, random_state=42)
metrics = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

best_val_loss = float('inf')
best_model_path = os.path.join(output_dir, 'best_model.pth')
best_epoch = -1
best_fold = -1

fold_val_true = []
fold_val_pred = []
fold_val_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(full_dataset)):
    print(f'Fold {fold + 1}')
    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
    
    model = build_model().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    fold_best_val_loss = float('inf')
    fold_best_epoch = -1
    fold_best_model_path = os.path.join(output_dir, f'best_model_fold_{fold + 1}.pth')

    for epoch in range(20):
        model.train()
        train_loss = 0.0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        model.eval()
        val_loss = 0.0
        val_true = []
        val_pred = []
        val_scores = []
        with torch.no_grad():
            for images, labels in tqdm(val_loader):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * images.size(0)
                val_true.extend(labels.cpu().numpy())
                val_pred.extend(outputs.argmax(dim=1).cpu().numpy())
                val_scores.extend(torch.softmax(outputs, dim=1).cpu().numpy())
        
        val_loss /= len(val_loader.dataset)
        
        if val_loss < fold_best_val_loss:
            fold_best_val_loss = val_loss
            fold_best_epoch = epoch
            torch.save(model.state_dict(), fold_best_model_path)
        
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    if fold_best_val_loss < best_val_loss:
        best_val_loss = fold_best_val_loss
        best_epoch = fold_best_epoch
        best_fold = fold + 1
        torch.save(model.state_dict(), best_model_path)
    
    model.load_state_dict(torch.load(fold_best_model_path))
    val_true = np.array(val_true)
    val_pred = np.array(val_pred)
    val_scores = np.array(val_scores)

    fold_val_true.extend(val_true)
    fold_val_pred.extend(val_pred)
    fold_val_scores.extend(val_scores)
    
    accuracy = accuracy_score(val_true, val_pred)
    precision = precision_score(val_true, val_pred, average='weighted')
    recall = recall_score(val_true, val_pred, average='weighted')
    f1 = f1_score(val_true, val_pred, average='weighted')
    auc = roc_auc_score(val_true, val_scores, multi_class='ovo')
    
    print(f'Best model found at epoch {fold_best_epoch}')
    print(f'Val Loss: {fold_best_val_loss:.4f} Acc: {accuracy:.4f} Precision: {precision:.4f} Recall: {recall:.4f} F1: {f1:.4f} AUC: {auc:.4f}')
    
    class_metrics = {}
    for class_idx, class_name in enumerate(full_dataset.classes):
        class_true = (val_true == class_idx).astype(int)
        class_pred = (val_pred == class_idx).astype(int)
        class_score = val_scores[:, class_idx]
        
        class_accuracy = accuracy_score(class_true, class_pred)
        class_precision = precision_score(class_true, class_pred)
        class_recall = recall_score(class_true, class_pred)
        class_f1 = f1_score(class_true, class_pred)
        class_auc = roc_auc_score(class_true, class_score)
        
        class_metrics[class_name] = {
            'accuracy': class_accuracy,
            'precision': class_precision,
            'recall': class_recall,
            'f1': class_f1,
            'auc': class_auc
        }
        
        print(f'Val Class {class_name} Acc: {class_accuracy:.4f} Precision: {class_precision:.4f} Recall: {class_recall:.4f} F1: {class_f1:.4f} AUC: {class_auc:.4f}')
    
    metrics.append({'fold': fold + 1, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1, 'auc': auc, 'class_metrics': class_metrics, 'best_epoch': fold_best_epoch})

# 保存五折交叉验证的指标
metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv(os.path.join(output_dir, 'cross_validation_metrics.csv'), index=False)

# 绘制五折交叉验证的ROC曲线
plt.figure()
for class_idx, class_name in enumerate(full_dataset.classes):
    class_true = (np.array(fold_val_true) == class_idx).astype(int)
    class_score = np.array(fold_val_scores)[:, class_idx]
    
    fpr, tpr, _ = roc_curve(class_true, class_score)
    auc = roc_auc_score(class_true, class_score)
    
    plt.plot(fpr, tpr, label=f'{class_name} (area = {auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Cross-Validation ROC Curve')
plt.legend(loc="lower right")
plt.savefig(os.path.join(output_dir, 'cross_validation_roc_curve.pdf'))
plt.close()

# 绘制五折交叉验证的混淆矩阵
cm = confusion_matrix(fold_val_true, fold_val_pred)
plt.figure()
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Cross-Validation Confusion Matrix')
plt.savefig(os.path.join(output_dir, 'cross_validation_confusion_matrix.pdf'))
plt.close()

# 加载最终最优模型并评估独立验证集
model.load_state_dict(torch.load(best_model_path))
model.eval()

val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
val_true = []
val_pred = []
val_scores = []
with torch.no_grad():
    for images, labels in tqdm(val_loader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        val_true.extend(labels.cpu().numpy())
        val_pred.extend(outputs.argmax(dim=1).cpu().numpy())
        val_scores.extend(torch.softmax(outputs, dim=1).cpu().numpy())

val_true = np.array(val_true)
val_pred = np.array(val_pred)
val_scores = np.array(val_scores)

# 计算独立验证集的指标
accuracy = accuracy_score(val_true, val_pred)
precision = precision_score(val_true, val_pred, average='weighted')
recall = recall_score(val_true, val_pred, average='weighted')
f1 = f1_score(val_true, val_pred, average='weighted')
auc = roc_auc_score(val_true, val_scores, multi_class='ovo')

print(f'Validation Set Metrics:')
print(f'Acc: {accuracy:.4f} Precision: {precision:.4f} Recall: {recall:.4f} F1: {f1:.4f} AUC: {auc:.4f}')

# 保存独立验证集的指标
val_metrics = {
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1,
    'auc': auc,
    'class_metrics': {}
}

for class_idx, class_name in enumerate(val_dataset.classes):
    class_true = (val_true == class_idx).astype(int)
    class_pred = (val_pred == class_idx).astype(int)
    class_score = val_scores[:, class_idx]
    
    class_accuracy = accuracy_score(class_true, class_pred)
    class_precision = precision_score(class_true, class_pred)
    class_recall = recall_score(class_true, class_pred)
    class_f1 = f1_score(class_true, class_pred)
    class_auc = roc_auc_score(class_true, class_score)
    
    val_metrics['class_metrics'][class_name] = {
        'accuracy': class_accuracy,
        'precision': class_precision,
        'recall': class_recall,
        'f1': class_f1,
        'auc': class_auc
    }
    
    print(f'Val Class {class_name} Acc: {class_accuracy:.4f} Precision: {class_precision:.4f} Recall: {class_recall:.4f} F1: {class_f1:.4f} AUC: {class_auc:.4f}')

# 保存独立验证集的指标
with open(os.path.join(output_dir, 'validation_metrics.csv'), 'w') as f:
    for key in val_metrics.keys():
        f.write(f"{key},{val_metrics[key]}\n")

# 绘制独立验证集的ROC曲线
plt.figure()
for class_idx, class_name in enumerate(val_dataset.classes):
    class_true = (val_true == class_idx).astype(int)
    class_score = val_scores[:, class_idx]
    
    fpr, tpr, _ = roc_curve(class_true, class_score)
    auc = roc_auc_score(class_true, class_score)
    
    plt.plot(fpr, tpr, label=f'{class_name} (area = {auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Validation ROC Curve')
plt.legend(loc="lower right")
plt.savefig(os.path.join(output_dir, 'validation_roc_curve.pdf'))
plt.close()

# 绘制独立验证集的混淆矩阵
cm = confusion_matrix(val_true, val_pred)
plt.figure()
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Validation Confusion Matrix')
plt.savefig(os.path.join(output_dir, 'validation_confusion_matrix.pdf'))
plt.close()

# 输出最终最优模型的折数和最佳轮次
print(f'The best model was found in fold {best_fold} at epoch {best_epoch}')


Fold 1


100%|██████████| 21/21 [00:26<00:00,  1.28s/it]


Epoch 1, Train Loss: 0.8946, Val Loss: 0.9740


100%|██████████| 21/21 [00:24<00:00,  1.18s/it]


Epoch 2, Train Loss: 0.8101, Val Loss: 1.4979


...

Epoch 19, Train Loss: 0.2101, Val Loss: 1.1213


100%|██████████| 21/21 [00:16<00:00,  1.25it/s]


Epoch 20, Train Loss: 0.1389, Val Loss: 2.7498
Best model found at epoch 7
Val Loss: 0.7712 Acc: 0.5745 Precision: 0.6493 Recall: 0.5745 F1: 0.5483 AUC: 0.8136
Val Class benign Acc: 0.7340 Precision: 0.6364 Recall: 0.2500 F1: 0.3590 AUC: 0.6702
Val Class malignant Acc: 0.6657 Precision: 0.4719 Recall: 0.9797 F1: 0.6370 AUC: 0.9094
Val Class normal Acc: 0.7492 Precision: 0.7907 Recall: 0.5132 F1: 0.6224 AUC: 0.8452


100%|██████████| 35/35 [00:32<00:00,  1.09it/s]


Validation Set Metrics:
Acc: 0.6347 Precision: 0.6451 Recall: 0.6347 F1: 0.6368 AUC: 0.7762
Val Class benign Acc: 0.6863 Precision: 0.6230 Recall: 0.5308 F1: 0.5732 AUC: 0.7240
Val Class malignant Acc: 0.8698 Precision: 0.3660 Recall: 0.5437 F1: 0.4375 AUC: 0.8212
Val Class normal Acc: 0.7134 Precision: 0.7133 Recall: 0.7323 F1: 0.7227 AUC: 0.7762
The best model was found in fold 4 at epoch 10
